# Evolver Loop 4 - LB Feedback Analysis

## Key Insight from Evaluator
The evaluator correctly identified that we've only used 4 pre-optimized sources, but there are 15+ available in the snapshots directory. The Jonathan Chan kernel achieves better scores by ensembling from many more sources.

## Plan
1. Comprehensively scan ALL pre-optimized CSVs
2. Score each one and identify best per-N
3. Build ensemble picking best per-N from ALL sources
4. Analyze which N values have room for improvement

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
import glob
import os
from tqdm import tqdm

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    vertices = [(tx * cos_a - ty * sin_a + x, tx * sin_a + ty * cos_a + y) for tx, ty in zip(TX, TY)]
    return Polygon(vertices)

def compute_bounding_side(polygons):
    if not polygons:
        return 0
    all_points = []
    for poly in polygons:
        all_points.extend(list(poly.exterior.coords))
    all_points = np.array(all_points)
    return max(all_points.max(axis=0) - all_points.min(axis=0))

def compute_score_for_n(df, n):
    prefix = f"{n:03d}_"
    trees = df[df['id'].str.startswith(prefix)]
    if len(trees) != n:
        return float('inf')
    polygons = [create_tree_polygon(parse_value(row['x']), parse_value(row['y']), parse_value(row['deg'])) for _, row in trees.iterrows()]
    side = compute_bounding_side(polygons)
    return side**2 / n

def compute_total_score(df):
    return sum(compute_score_for_n(df, n) for n in range(1, 201))

print("Functions defined")

Functions defined


In [2]:
# Scan ALL pre-optimized CSVs
base_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized'

# Find all CSV files recursively
all_csvs = glob.glob(f'{base_path}/**/*.csv', recursive=True)
print(f"Found {len(all_csvs)} CSV files")

# Also check external data
external_csvs = glob.glob('/home/code/external_data/**/*.csv', recursive=True)
print(f"Found {len(external_csvs)} external CSV files")

all_csvs.extend(external_csvs)
print(f"Total: {len(all_csvs)} CSV files to evaluate")

Found 30 CSV files
Found 7 external CSV files
Total: 37 CSV files to evaluate


In [3]:
# Load and score each CSV
scores = {}
per_n_scores = {}  # {source: {n: score}}

for csv_path in tqdm(all_csvs):
    try:
        df = pd.read_csv(csv_path)
        if 'id' not in df.columns or 'x' not in df.columns:
            continue
        
        # Check if it has all N values
        total_score = compute_total_score(df)
        if total_score < float('inf'):
            source_name = csv_path.replace(base_path, '').replace('/home/code/external_data/', 'ext:')
            scores[source_name] = total_score
            
            # Compute per-N scores
            per_n_scores[source_name] = {}
            for n in range(1, 201):
                per_n_scores[source_name][n] = compute_score_for_n(df, n)
    except Exception as e:
        print(f"Error loading {csv_path}: {e}")

print(f"\nSuccessfully scored {len(scores)} CSV files")

  0%|          | 0/37 [00:00<?, ?it/s]

  3%|▎         | 1/37 [00:03<02:19,  3.89s/it]

  5%|▌         | 2/37 [00:07<02:16,  3.89s/it]

  8%|▊         | 3/37 [00:11<02:13,  3.92s/it]

 11%|█         | 4/37 [00:15<02:09,  3.92s/it]

...

 95%|█████████▍| 35/37 [02:16<00:07,  3.92s/it]

 97%|█████████▋| 36/37 [02:20<00:03,  3.92s/it]

100%|██████████| 37/37 [02:24<00:00,  3.92s/it]

100%|██████████| 37/37 [02:24<00:00,  3.90s/it]


Successfully scored 37 CSV files


In [ ]:
# Sort by score
sorted_scores = sorted(scores.items(), key=lambda x: x[1])
print("Top 20 pre-optimized solutions:")
print("="*60)
for i, (source, score) in enumerate(sorted_scores[:20]):
    print(f"{i+1:2d}. {score:.6f} - {source}")

print(f"\nBest score: {sorted_scores[0][1]:.6f}")
print(f"Target: 68.919154")
print(f"Gap: {sorted_scores[0][1] - 68.919154:.6f}")

In [ ]:
# Build ensemble picking best per-N from ALL sources
print("\nBuilding comprehensive ensemble...")

best_per_n = {}  # {n: (best_score, best_source)}
for n in range(1, 201):
    best_score = float('inf')
    best_source = None
    for source, n_scores in per_n_scores.items():
        if n in n_scores and n_scores[n] < best_score:
            best_score = n_scores[n]
            best_source = source
    best_per_n[n] = (best_score, best_source)

# Count wins per source
wins_per_source = {}
for n, (score, source) in best_per_n.items():
    wins_per_source[source] = wins_per_source.get(source, 0) + 1

print("\nWins per source:")
for source, wins in sorted(wins_per_source.items(), key=lambda x: -x[1]):
    print(f"  {wins:3d} wins: {source}")

# Calculate ensemble score
ensemble_score = sum(score for score, _ in best_per_n.values())
print(f"\nEnsemble score (best per-N from all sources): {ensemble_score:.6f}")
print(f"Best single source: {sorted_scores[0][1]:.6f}")
print(f"Improvement from ensemble: {sorted_scores[0][1] - ensemble_score:.6f}")

In [ ]:
# Analyze which N values have room for improvement
print("\nPer-N analysis - looking for improvement opportunities:")
print("="*70)

# Get best source's per-N scores
best_source = sorted_scores[0][0]
best_source_scores = per_n_scores[best_source]

# Compare to ensemble
print("N values where ensemble beats best single source:")
improvements = []
for n in range(1, 201):
    best_single = best_source_scores[n]
    ensemble_n = best_per_n[n][0]
    if ensemble_n < best_single - 1e-9:
        improvements.append((n, best_single - ensemble_n, best_per_n[n][1]))
        print(f"  N={n}: {best_single:.6f} -> {ensemble_n:.6f} (improvement: {best_single - ensemble_n:.6f}) from {best_per_n[n][1]}")

if not improvements:
    print("  None - best single source wins for all N values")
else:
    print(f"\nTotal improvements: {len(improvements)} N values")
    print(f"Total score improvement: {sum(imp[1] for imp in improvements):.6f}")

In [ ]:
# Look at theoretical optimal efficiency
print("\nEfficiency analysis (score_n / n):")
print("="*60)

efficiencies = []
for n in range(1, 201):
    score_n = best_per_n[n][0]
    efficiency = score_n  # Already normalized by n
    efficiencies.append((n, efficiency))

# Sort by efficiency (worst first)
efficiencies.sort(key=lambda x: -x[1])

print("Worst 20 N values (most room for improvement):")
for n, eff in efficiencies[:20]:
    print(f"  N={n:3d}: efficiency={eff:.6f}")

print("\nBest 20 N values:")
for n, eff in efficiencies[-20:]:
    print(f"  N={n:3d}: efficiency={eff:.6f}")

In [ ]:
# Create the ensemble submission
print("\nCreating ensemble submission...")

# Load all source dataframes
source_dfs = {}
for source in wins_per_source.keys():
    if source.startswith('ext:'):
        path = '/home/code/external_data/' + source[4:]
    else:
        path = base_path + source
    source_dfs[source] = pd.read_csv(path)

# Build ensemble
ensemble_rows = []
for n in range(1, 201):
    best_score, best_source = best_per_n[n]
    df = source_dfs[best_source]
    prefix = f"{n:03d}_"
    trees = df[df['id'].str.startswith(prefix)]
    for _, row in trees.iterrows():
        ensemble_rows.append(row.to_dict())

ensemble_df = pd.DataFrame(ensemble_rows)
ensemble_df.to_csv('/home/submission/submission.csv', index=False)
print(f"Saved ensemble with {len(ensemble_df)} rows")

# Verify
df_verify = pd.read_csv('/home/submission/submission.csv')
verify_score = compute_total_score(df_verify)
print(f"Verified ensemble score: {verify_score:.6f}")
print(f"Target: 68.919154")
print(f"Gap: {verify_score - 68.919154:.6f}")